In [31]:
import pandas as pd
from datetime import date, timedelta

In [32]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [33]:
df.tail()

,diagnosis_date,agegroup
677917,2022-03-06,60-69
677918,2022-03-06,50-59
677919,2022-03-06,40-49
677920,2022-03-06,10-19_
677921,2022-03-06,10-19_


In [34]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [35]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-03-03,296,504,452,363,357,213,126,67,25,4
2022-03-04,225,433,325,304,313,187,124,64,20,5
2022-03-05,195,445,297,264,293,207,100,60,29,2
2022-03-06,179,345,210,199,245,140,100,75,24,7
2022-03-07,200,376,195,200,280,155,83,48,39,15


In [36]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

1591

In [37]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [38]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
89,2022-03-04,296,504
90,2022-03-05,225,433
91,2022-03-06,195,445
92,2022-03-07,179,345
93,2022-03-08,200,376


In [39]:
df_0_19.to_csv("../data/cases_0_19.csv")